In [2]:
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
import numpy as np
import wget

In [3]:
def atypical_xml_process(url):
    response = requests.get(url)
    if response.status_code == 200:
        Data = BeautifulSoup(response.text, 'lxml-xml')
        allData = Data.find_all()
        
        tagLevelUp = 0
        tagLevelDown = 0
        
        # 태그의 레벨을 기록할 딕셔너리 생성
        recordDict = {}
        
        
        ### xml 데이터의 모든 태그를 검색하여 각 태그의 단계를 기록
        ## xml 데이터에 존재하는 모든 태그에 대해 반복
        for findTagIndex in range(0, len(allData)):
            # 찾을 태그명
            findTagName = allData[findTagIndex].name
            
            ## 이미 검색된 태그인지 검사
            alreadySearched = False
            for eachDictKeyIndex in range(0, len(recordDict)):
                eachDictValues = recordDict[eachDictKeyIndex]
                for eachDictValueIndex in range(0, len(eachDictValues)):
                    if (findTagName == eachDictValues[eachDictValueIndex]):
                        break
                if (findTagName == eachDictValues[eachDictValueIndex]):
                    alreadySearched = True
                    break
            
            ## 이미 검색된 태그에 대해서 상위태그 검색 생략
            if alreadySearched == True:
                continue
            
            ## 찾을 태그의 차상위 단계 부터 최상위 단계 까지 반복
            for upperTagIndex in range((findTagIndex - 1), -1, -1):
                # 데이터의 상위태그명
                upperTag = allData[upperTagIndex]
                upperTagName = upperTag.name
                # 상위 태그에 존재하는 모든 하위태그
                findLowerTag = upperTag.find_all()

                # 하위태그의 수만큼 반복
                for lowerTagIndex in range(0, len(findLowerTag)):
                    # 각 하위태그명
                    lowerTagName = findLowerTag[lowerTagIndex].name
                    # 찾을 값이 어느 상위태그 아래에 있는지 검색
                    if (findTagName == lowerTagName):
                        break
                # 상위태그보다 1단계 위의 레벨 기록
                if (findTagName == lowerTagName):
                    tagLevelUp = upperTagIndex + 1
                    break

            # 태그가 속한 레벨
            tagLevel = tagLevelUp - tagLevelDown

            ## 차상위 태그가 존재하는지 검색 (예외적인 태그 구성에 대한 조치)
            try:
                # 차상위 태그가 없을 경우 KeyError 발생
                checkError = recordDict[tagLevel - 1]
            except:
                # 현재까지 만들어진 딕셔너리의 값
                for currentEachValueIndex in range(0, len(recordDict)):
                    eachDictValues = recordDict[currentEachValueIndex]
                    # 차상위 태그가 위치한 레벨을 검색
                    for eachDictValueIndex in range(0, len(eachDictValues)):
                        eachDictValue = eachDictValues[eachDictValueIndex]
                        if eachDictValue == upperTagName:
                            break
                    if (eachDictValue == upperTagName):
                        tagLevelDown = tagLevelUp - (currentEachValueIndex + 1)
                        break
                # 올바른 태그 레벨 반환
                tagLevel = tagLevelUp - tagLevelDown

            try:
                # 딕셔너리의 해당 태그 레벨에 값 추가
                recordDict[tagLevel].append(findTagName)
            except:
                # 해당 태그 레벨에 값이 없을 경우 값 넣기
                recordDict[tagLevel] = [findTagName]
        
        if (len(recordDict) == 0):
            print("No Tag Searched")
            return None
        
        ### 생성된 딕셔너리에서 itemList와 separatorTag 검색
        findMaxItemList = []
        for eachDictKeyIndex in range(0, len(recordDict)):
            # 딕셔너리의 각 값의 길이를 리스트에 추가
            findMaxItemList.append(len(recordDict[eachDictKeyIndex]))
            
        # 길이가 가장 긴 item 기록
        maxItemKinds = max(findMaxItemList)
        
        if len(recordDict) > 1:
            for eachDictKeyIndex in range(1, len(recordDict)):
                # 딕셔너리의 상위레벨부터 max값과 비교
                if (maxItemKinds == len(recordDict[eachDictKeyIndex])):
                    # 해당 값들을 itemList에 기록
                    itemList = recordDict[eachDictKeyIndex]
                    # 해당 값들을 포함하는 차상위 태그를 separatorTag에 선언
                    for upperKeyIndex in range((eachDictKeyIndex - 1), -1, -1):
                        if (len(recordDict[upperKeyIndex]) == 1):
                            upperTagValue = recordDict[upperKeyIndex]
                            break
                        else:
                            continue
                    try:
                        separatorTag = upperTagValue[0]
                    except:
                        separatorTag = recordDict[0][0]
                    break
        else:
            print('Item Level is Only One')
            iterData = Data.find_all(recordDict[0][0])
            for dataRow in range(0, len(iterData)):
                columnName = iterData[0].name
                resultData = iterData[dataRow].text
            finalResult = pd.DataFrame(data = [resultData], columns = [columnName])
            return finalResult
        
        
        ### 데이터 프레임을 만들 최종 딕셔너리 생성
        # 값을 담을 딕셔너리 공간 생성
        resultDict = {}
        # 딕셔너리의 각 키를 itemList로 생성하며, 각 값을 넣을 공간을 리스트로 생성
        for eachItem in itemList:
            resultDict[eachItem] = []
        iterData = Data.find_all(separatorTag)
        # 데이터의 길이만큼 반복(row)
        for dataRow in range(0, len(iterData)):
            # itemList만큼 반복(column)
            for dataColumn in range(0, len(itemList)):
                eachData = iterData[dataRow].find_all(itemList[dataColumn])
                # 각 값을 담을 리스트 생성 후 값 담기
                eachItemList = []
                for eachText in range(0, len(eachData)):
                    tag = eachData[eachText]
                    eachItemList.append(tag.text)
                
                # 결측값 조치
                if (len(eachItemList) == 0):
                    resultDict[itemList[dataColumn]].append('')
                # 값이 있을 경우 추가
                elif (len(eachItemList) == 1):
                    resultDict[itemList[dataColumn]] += eachItemList
                # 값이 2개 이상 있을 경우 리스트로 묶어서 추가
                else:
                    resultDict[itemList[dataColumn]].append(eachItemList)
        ### 데이터 프레임을 만들고 return
        finalResult = pd.DataFrame(resultDict, columns = itemList)
        return finalResult
    
    else:
        print('Bad Response', response.status_code)
        return None

In [ ]:
def atypical_xml_process(url):
    response = requests.get(url)
    if response.status_code == 200:
        Data = BeautifulSoup(response.text, 'lxml-xml')
        allData = Data.find_all()
        
        tagLevelUp = 0
        tagLevelDown = 0
        
        # 태그의 레벨을 기록할 딕셔너리 생성
        recordDict = {}
        
        
        ### xml 데이터의 모든 태그를 검색하여 각 태그의 단계를 기록
        ## xml 데이터에 존재하는 모든 태그에 대해 반복
        for findTagIndex in range(0, len(allData)):
            # 찾을 태그명
            findTagName = allData[findTagIndex].name
            
            ## 이미 검색된 태그인지 검사
            alreadySearched = False
            for eachDictKeyIndex in range(0, len(recordDict)):
                eachDictValues = recordDict[eachDictKeyIndex]
                for eachDictValueIndex in range(0, len(eachDictValues)):
                    if (findTagName == eachDictValues[eachDictValueIndex]):
                        break
                if (findTagName == eachDictValues[eachDictValueIndex]):
                    alreadySearched = True
                    break
            
            ## 이미 검색된 태그에 대해서 상위태그 검색 생략
            if alreadySearched == True:
                continue
            
            ## 찾을 태그의 차상위 단계 부터 최상위 단계 까지 반복
            for upperTagIndex in range((findTagIndex - 1), -1, -1):
                # 데이터의 상위태그명
                upperTag = allData[upperTagIndex]
                upperTagName = upperTag.name
                # 상위 태그에 존재하는 모든 하위태그
                findLowerTag = upperTag.find_all()

                # 하위태그의 수만큼 반복
                for lowerTagIndex in range(0, len(findLowerTag)):
                    # 각 하위태그명
                    lowerTagName = findLowerTag[lowerTagIndex].name
                    # 찾을 값이 어느 상위태그 아래에 있는지 검색
                    if (findTagName == lowerTagName):
                        break
                # 상위태그보다 1단계 위의 레벨 기록
                if (findTagName == lowerTagName):
                    tagLevelUp = upperTagIndex + 1
                    break

            # 태그가 속한 레벨
            tagLevel = tagLevelUp - tagLevelDown

            ## 차상위 태그가 존재하는지 검색 (예외적인 태그 구성에 대한 조치)
            try:
                # 차상위 태그가 없을 경우 KeyError 발생
                checkError = recordDict[tagLevel - 1]
            except:
                # 현재까지 만들어진 딕셔너리의 값
                for currentEachValueIndex in range(0, len(recordDict)):
                    eachDictValues = recordDict[currentEachValueIndex]
                    # 차상위 태그가 위치한 레벨을 검색
                    for eachDictValueIndex in range(0, len(eachDictValues)):
                        eachDictValue = eachDictValues[eachDictValueIndex]
                        if eachDictValue == upperTagName:
                            break
                    if (eachDictValue == upperTagName):
                        tagLevelDown = tagLevelUp - (currentEachValueIndex + 1)
                        break
                # 올바른 태그 레벨 반환
                tagLevel = tagLevelUp - tagLevelDown

            try:
                # 딕셔너리의 해당 태그 레벨에 값 추가
                recordDict[tagLevel].append(findTagName)
            except:
                # 해당 태그 레벨에 값이 없을 경우 값 넣기
                recordDict[tagLevel] = [findTagName]
        
        if (len(recordDict) == 0):
            print("No Tag Searched")
            return None
        
        ### 생성된 딕셔너리에서 itemList와 separatorTag 검색
        findMaxItemList = []
        for eachDictKeyIndex in range(0, len(recordDict)):
            # 딕셔너리의 각 값의 길이를 리스트에 추가
            findMaxItemList.append(len(recordDict[eachDictKeyIndex]))
            
        # 길이가 가장 긴 item 기록
        maxItemKinds = max(findMaxItemList)
        
        if len(recordDict) > 1:
            for eachDictKeyIndex in range(1, len(recordDict)):
                # 딕셔너리의 상위레벨부터 max값과 비교
                if (maxItemKinds == len(recordDict[eachDictKeyIndex])):
                    # 해당 값들을 itemList에 기록
                    itemList = recordDict[eachDictKeyIndex]
                    # 해당 값들을 포함하는 차상위 태그를 separatorTag에 선언
                    for upperKeyIndex in range((eachDictKeyIndex - 1), -1, -1):
                        if (len(recordDict[upperKeyIndex]) == 1):
                            upperTagValue = recordDict[upperKeyIndex]
                            break
                        else:
                            continue
                    try:
                        separatorTag = upperTagValue[0]
                    except:
                        separatorTag = recordDict[0][0]
                    break
        else:
            print('Item Level is Only One')
            iterData = Data.find_all(recordDict[0][0])
            for dataRow in range(0, len(iterData)):
                columnName = iterData[0].name
                resultData = iterData[dataRow].text
            finalResult = pd.DataFrame(data = [resultData], columns = [columnName])
            return finalResult
        
        
        ### 데이터 프레임을 만들 최종 딕셔너리 생성
        # 값을 담을 딕셔너리 공간 생성
        resultDict = {}
        # 딕셔너리의 각 키를 itemList로 생성하며, 각 값을 넣을 공간을 리스트로 생성
        for eachItem in itemList:
            resultDict[eachItem] = []
        iterData = Data.find_all(separatorTag)
        # 데이터의 길이만큼 반복(row)
        for dataRow in range(0, len(iterData)):
            # itemList만큼 반복(column)
            for dataColumn in range(0, len(itemList)):
                eachData = iterData[dataRow].find_all(itemList[dataColumn])
                # 각 값을 담을 리스트 생성 후 값 담기
                eachItemList = []
                for eachText in range(0, len(eachData)):
                    tag = eachData[eachText]
                    eachItemList.append(tag.text)
                
                # 결측값 조치
                if (len(eachItemList) == 0):
                    resultDict[itemList[dataColumn]].append('')
                # 값이 있을 경우 추가
                elif (len(eachItemList) == 1):
                    resultDict[itemList[dataColumn]] += eachItemList
                # 값이 2개 이상 있을 경우 리스트로 묶어서 추가
                else:
                    resultDict[itemList[dataColumn]].append(eachItemList)
        ### 데이터 프레임을 만들고 return
        finalResult = pd.DataFrame(resultDict, columns = itemList)
        return finalResult
    
    else:
        print('Bad Response', response.status_code)
        return None

In [3]:
url = 'http://apis.data.go.kr/1611000/nsdi/BuildingUseService/attr/getBuildingUse?ServiceKey=aDTqxJKNQ6XA6akRhrCF1ZSGXu8uk6HwyjXEWNetiKZEo%2FAD1M38g97KxPIXwL5k4RBnx%2BJwa6DzpTgn0G6AEg%3D%3D&pnu=1111017400'
atypical_xml_process(url)

,gisIdntfcNo,pnu,ldCode,ldCodeNm,regstrSeCode,regstrSeCodeNm,mnnmSlno,buldIdntfcNo,agbldgSeCode,agbldgSeCodeNm,...,detailPrposCodeNm,buldPrposClCode,buldPrposClCodeNm,buldHg,groundFloorCo,undgrndFloorCo,prmisnDe,useConfmDe,lastUpdtDt,buldDongNm
0,1988201247664531326300000000,1111017400100010001,1111017400,서울특별시 종로구 창신동,1,일반,1-1,21483,1,일반건축물,...,기타제2종근생활시설,2,상업용,0,2,1,1988-03-15,1988-08-05,2019-01-02,
1,1988201248874531224000000000,1111017400100010082,1111017400,서울특별시 종로구 창신동,1,일반,1-82,14204,1,일반건축물,...,기타사무소,2,상업용,8.6,3,1,1987-12-28,1988-08-18,2019-01-02,창신동주택
2,1984201253274531009800000000,1111017400100010084,1111017400,서울특별시 종로구 창신동,1,일반,1-84,26646,1,일반건축물,...,기타제2종근생활시설,2,상업용,0,3,1,1983-12-09,1984-11-28,2019-01-02,
3,1944201215344530567700000000,1111017400100010091,1111017400,서울특별시 종로구 창신동,1,일반,1-91,29289,1,일반건축물,...,단독주택,1,주거용,0,1,0,,1944-09-01,2019-01-02,
4,1983201255034530735400000000,1111017400100020002,1111017400,서울특별시 종로구 창신동,1,일반,2-2,24748,1,일반건축물,...,기타제2종근생활시설,2,상업용,0,2,0,1983-03-22,1983-07-28,2019-01-02,
5,1998201250684530852400000000,1111017400100030003,1111017400,서울특별시 종로구 창신동,1,일반,3-3,4168,1,일반건축물,...,사무소,2,상업용,24.85,7,2,1996-08-06,1998-08-03,2019-01-02,
6,1995201204284530524200000000,1111017400100060015,1111017400,서울특별시 종로구 창신동,1,일반,6-15,20229,1,일반건축물,...,기타제2종근생활시설,2,상업용,9.8,3,1,1994-08-19,1995-01-25,2019-01-02,
7,0000201196244530527000000000,1111017400100060016,1111017400,서울특별시 종로구 창신동,1,일반,6-16,15104,1,일반건축물,...,단독주택,1,주거용,0,1,0,,,2019-01-02,
8,1944201187434530535300000000,1111017400100060017,1111017400,서울특별시 종로구 창신동,1,일반,6-17,19386,1,일반건축물,...,단독주택,1,주거용,0,1,0,,1944-09-10,2019-01-02,
9,0000201185364530558700000000,1111017400100060017,1111017400,서울특별시 종로구 창신동,1,일반,6-17,19387,1,일반건축물,...,기타제1종근생활시설,2,상업용,0,1,0,,,2019-01-02,제1호


In [4]:
url = 'http://apis.data.go.kr/1611000/nsdi/BuildingUseService/wfs/getBuildingUseWFS?ServiceKey=aDTqxJKNQ6XA6akRhrCF1ZSGXu8uk6HwyjXEWNetiKZEo%2FAD1M38g97KxPIXwL5k4RBnx%2BJwa6DzpTgn0G6AEg%3D%3D'
testData = atypical_xml_process(url)

In [5]:
testData.to_csv("./testData.csv", encoding='ms949', index=False)

In [44]:
a = {'1' : 1}

In [51]:
list(a.keys())[0]

'1'

In [33]:
pd.DataFrame(data = [''])

,0
0,


In [7]:
tempData = '<?xml version="1.0" encoding="UTF-8"?>\
<wfs:FeatureCollection1 xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:wfs="http://www.opengis.net/wfs" xmlns:gml="http://www.opengis.net/gml" xmlns:NSDI="http://openapi.nsdi.go.kr" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" numberOfFeatures="10" timeStamp="2019-04-27T16:24:01.636Z" xsi:schemaLocation="http://openapi.nsdi.go.kr http://10.20.10.46:8080/geoserver/NSDI/wfs?service=WFS&amp;version=1.1.0&amp;request=DescribeFeatureType&amp;typeName=NSDI%3AF253 http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.1.0/wfs.xsd">\
217853.568 448978.337 217841.468 448975.857 217842.798 448971.337 217852.098 448972.997 217853.938 448975.767 217853.568 448978.337\
</wfs:FeatureCollection1>'
Data = BeautifulSoup(tempData, 'lxml-xml')
allData = Data.find_all()

tagLevelUp = 0
tagLevelDown = 0

# 태그의 레벨을 기록할 딕셔너리 생성
recordDict = {}


### xml 데이터의 모든 태그를 검색하여 각 태그의 단계를 기록
## xml 데이터에 존재하는 모든 태그에 대해 반복
for findTagIndex in range(0, len(allData)):
    # 찾을 태그명
    findTagName = allData[findTagIndex].name

    ## 이미 검색된 태그인지 검사
    alreadySearched = False
    for eachDictKeyIndex in range(0, len(recordDict)):
        eachDictValues = recordDict[eachDictKeyIndex]
        for eachDictValueIndex in range(0, len(eachDictValues)):
            if (findTagName == eachDictValues[eachDictValueIndex]):
                break
        if (findTagName == eachDictValues[eachDictValueIndex]):
            alreadySearched = True
            break

    ## 이미 검색된 태그에 대해서 상위태그 검색 생략
    if alreadySearched == True:
        continue

    ## 찾을 태그의 차상위 단계 부터 최상위 단계 까지 반복
    for upperTagIndex in range((findTagIndex - 1), -1, -1):
        # 데이터의 상위태그명
        upperTag = allData[upperTagIndex]
        upperTagName = upperTag.name
        # 상위 태그에 존재하는 모든 하위태그
        findLowerTag = upperTag.find_all()

        # 하위태그의 수만큼 반복
        for lowerTagIndex in range(0, len(findLowerTag)):
            # 각 하위태그명
            lowerTagName = findLowerTag[lowerTagIndex].name
            # 찾을 값이 어느 상위태그 아래에 있는지 검색
            if (findTagName == lowerTagName):
                break
        # 상위태그보다 1단계 위의 레벨 기록
        if (findTagName == lowerTagName):
            tagLevelUp = upperTagIndex + 1
            break

    # 태그가 속한 레벨
    tagLevel = tagLevelUp - tagLevelDown

    ## 차상위 태그가 존재하는지 검색 (예외적인 태그 구성에 대한 조치)
    try:
        # 차상위 태그가 없을 경우 KeyError 발생
        checkError = recordDict[tagLevel - 1]
    except:
        # 현재까지 만들어진 딕셔너리의 값
        for currentEachValueIndex in range(0, len(recordDict)):
            eachDictValues = recordDict[currentEachValueIndex]
            # 차상위 태그가 위치한 레벨을 검색
            for eachDictValueIndex in range(0, len(eachDictValues)):
                eachDictValue = eachDictValues[eachDictValueIndex]
                if eachDictValue == upperTagName:
                    break
            if (eachDictValue == upperTagName):
                tagLevelDown = tagLevelUp - (currentEachValueIndex + 1)
                break
        # 올바른 태그 레벨 반환
        tagLevel = tagLevelUp - tagLevelDown

    try:
        # 딕셔너리의 해당 태그 레벨에 값 추가
        recordDict[tagLevel].append(findTagName)
    except:
        # 해당 태그 레벨에 값이 없을 경우 값 넣기
        recordDict[tagLevel] = [findTagName]

print(recordDict)
### 생성된 딕셔너리에서 itemList와 separatorTag 검색
findMaxItemList = []
for eachDictKeyIndex in range(0, len(recordDict)):
    # 딕셔너리의 각 값의 길이를 리스트에 추가
    findMaxItemList.append(len(recordDict[eachDictKeyIndex]))

# 길이가 가장 긴 item 기록
maxItemKinds = max(findMaxItemList)
print(findMaxItemList, maxItemKinds)

if len(recordDict) > 1:
    for eachDictKeyIndex in range(1, len(recordDict)):
        # 딕셔너리의 상위레벨부터 max값과 비교
        if (maxItemKinds == len(recordDict[eachDictKeyIndex])):
            # 해당 값들을 itemList에 기록
            itemList = recordDict[eachDictKeyIndex]
            # 해당 값들을 포함하는 차상위 태그를 separatorTag에 선언
            for upperKeyIndex in range((eachDictKeyIndex - 1), -1, -1):
                if (len(recordDict[upperKeyIndex]) > 1 & len(recordDict[upperKeyIndex]) == 0):
                    continue
                else:
                    upperTagValue = recordDict[upperKeyIndex]
                    break
            try:
                separatorTag = upperTagValue[0]
            except:
                separatorTag = recordDict[0][0]
            break
else:
    print('Item Level is Only One')
    iterData = Data.find_all(recordDict[0][0])
    for dataRow in range(0, len(iterData)):
        columnName = iterData[0].name
        resultData = iterData[dataRow].text
    finalResult = pd.DataFrame(data = [resultData], columns = [columnName])
# print(separatorTag, itemList)

# ### 데이터 프레임을 만들 최종 딕셔너리 생성
# # 값을 담을 딕셔너리 공간 생성
# resultDict = {}
# # 딕셔너리의 각 키를 itemList로 생성하며, 각 값을 넣을 공간을 리스트로 생성
# for eachItem in itemList:
#     resultDict[eachItem] = []
# iterData = Data.find_all(separatorTag)
# # 데이터의 길이만큼 반복(row)
# for dataRow in range(0, len(iterData)):
#     # itemList만큼 반복(column)
#     for dataColumn in range(0, len(itemList)):
#         eachData = iterData[dataRow].find_all(itemList[dataColumn])
#         # 각 값을 담을 리스트 생성 후 값 담기
#         eachItemList = []
#         for eachText in range(0, len(eachData)):
#             tag = eachData[eachText]
#             eachItemList.append(tag.text)
#         ## 각 키에 대한 값 추가
#         # 결측값은 공백으로 조치
#         if (len(eachItemList) == 0):
#             resultDict[itemList[dataColumn]].append('')
#         # 값이 있을 경우 추가
#         elif (len(eachItemList) == 1):
#             resultDict[itemList[dataColumn]] += eachItemList
#         # 값이 2개 이상 있을 경우 리스트로 묶어서 추가
#         else:
#             resultDict[itemList[dataColumn]].append(eachItemList)

# print(resultDict)

# ### 데이터 프레임을 만들고 return
# finalResult = pd.DataFrame(resultDict, columns = itemList)
# finalResult

{0: ['FeatureCollection1']}
[1] 1
Item Level is Only One


In [8]:
finalResult

,FeatureCollection1
0,217853.568 448978.337 217841.468 448975.857 21...


In [ ]:
# Backup
def atypical_xml_process(url):
    response = requests.get(url)
    if response.status_code == 200:
        Data = BeautifulSoup(response.text, 'lxml-xml')
        allData = Data.find_all()

        tagLevelUp = 0
        tagLevelDown = 0
        tagLevel = 0

        recordDict = {}

        for i in range(0, len(allData)):
            findTagName = allData[i].name
            for j in range(i, 0, -1):
                upperTagName = allData[j - 1].name
                findLowerTag = allData[j - 1].find_all()
                for k in range(0, len(findLowerTag)):
                    lowerTagName = findLowerTag[k].name
                    if (findTagName == lowerTagName):
                        break
                if (findTagName == lowerTagName):
                    tagLevelUp = j
                    break

            tagLevel = tagLevelUp - tagLevelDown

            try:
                checkError = recordDict[tagLevel - 1]
            except:
                for j in range(0, len(recordDict)):
                    eachDictValues = recordDict[j]
                    for k in range(0, len(eachDictValues)):
                        eachDictValue = eachDictValues[k]
                        if eachDictValue == upperTagName:
                            break
                    if (eachDictValue == upperTagName):
                        tagLevelDown = tagLevelUp - (j + 1)
                        break
                tagLevel = tagLevelUp - tagLevelDown

            try:
                overlapCheck = recordDict[tagLevel]
                isOverlap = False
                for j in range(0, len(overlapCheck)):
                    if overlapCheck[j] == findTagName:
                        isOverlap = True
                        break
                if (isOverlap == False):
                    recordDict[tagLevel].append(findTagName)
            except:
                recordDict[tagLevel] = [findTagName]

        findMaxItemList = []
        for i in range(0, len(recordDict)):
            findMaxItemList.append(len(recordDict[i]))
        maxItemNumber = max(findMaxItemList)
        for i in range(len(recordDict)):
            if (maxItemNumber == len(recordDict[i])):
                try:
                    upperTagValue = recordDict[i - 1]
                    itemList = recordDict[i]
                except:
                    upperTagValue = recordDict[0]
                    itemList = recordDict[1]
                separatorTag = upperTagValue[len(upperTagValue) - 1]
                break

        resultDict = {}
        for i in itemList:
            resultDict[i] = []
            print(resultDict)
        iterData = Data.find_all(separatorTag)
        print(iterData)
        for i in range(0, len(iterData)):
            for j in range(0, len(itemList)):
                eachData = iterData[i].find_all(itemList[j])
                print(eachData)
                eachItemList = []
                if eachData == []:
                    eachItemList.append('')
                else:
                    for k in range(0, len(eachData)):
                        tag = eachData[k]
                        eachItemList.append(tag.text)
                resultDict[itemList[j]] += eachItemList
        finalResult = pd.DataFrame(resultDict, columns = itemList)
        return finalResult
    
    else:
        print('Bad Response')
        return response.status_code